# CAPSTONE PROJECT- THE BATTLE OF THE NEIGHBORHOODS

## Week 2

### Diego González C.

In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install geopandas
!{sys.executable} -m pip install geopy

print("\nPackages installed succesfully.")

     |████████████████████████████████| 102kB 9.0MB/s ta 0:00:011
     |████████████████████████████████| 102kB 9.0MB/s ta 0:00:011
     |████████████████████████████████| 972kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 30.5MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 31.7MB/s eta 0:00:01

Packages installed succesfully.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('\nLibraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

### 1. A description of the problem and a discussion of the background.

Houston, Texas is one of the most important cities in the US. Its vast amout of food, sports and entertainment venues along with its medical and educational services make it a very attractive city to live. For this project I will make an analysis which will try to find the best neighborhoods to look for an apartment or house taking in consideration the following characteristics:

1. Atractive venues such as restaurants, bars, cafes and museums nearby. 
2. Public transportation service nearby
3. Hospitals or health care services nearby
4. Education services nearby (priority on universities)

### Interested Audience

I will attempt to construct a methodology which can be replied in any city in the world. Any person who is considering moving to a new city want to find the best location posible for the best price. For this matter, this tool will hopefully be useful for a large amount of people. 

### 2. A description of the data and how it will be used to solve the problem.

For the most part, the data that will be used will be obtained from Foursquare location data. The information that will be gathered from foursquare will be: Since foursquare gives the data from the venues, public transportation, hospitals and education centers this will be our most important source. 

* Attractive venues (Restaurants, bars, museums, etc.)
* Public transportation venues
* Hospitals 
* Colleges


### How data will be used to solve the problem

* Houston Neighborhoods will be clustered in groups, which will include the top 10 attractive venues from such neighborhoods using Foursquare and our geopy tools. 
* On the same way, the location of the public transportation stations, hospitals and schools will be obtained from Foursquare and geopy. Such venues will be clustered separately. 
* On the other hand, data of department/house rental prices and locations will be searched in open data sources: Government agencies, real estate agencies, etc.
* All of the above information will be depicted on a map using Folium to have a visual representation for better decision making. 


### Our decision for best neighborhood to live will consider:

* Are there attractive venues nearby?
* Are there any patterns for the distribution of popular venues and public services across Houston?
* Final recomendations from the information gathered. 

## INITIAL DATA - VENUES (FROM FOURSQUARE)

In [3]:

address = 'Houston, TX'

geolocator = Nominatim(user_agent="diego.gonzalez_13@hotmail.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinates of Houston, TX are 29.7589382, -95.3676974.


In [4]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: XXXXXXXXXXXXXXXXXXXXXX
CLIENT_SECRET: XXXXXXXXXXXXXXXXXXXXXX


In [5]:

LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WCU1ZCMQCCWCPEGMUS003YA4A5N0QWXRSUL0LSM4TVWO5O0U&client_secret=TWFVNVI05BPBMXXHGBR0GT2TLROCU3RVU0TBWCNHYJZJBSLA&v=20180605&ll=29.7589382,-95.3676974&radius=10000&limit=200'

In [6]:
results = requests.get(url).json()

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat','venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.head())


                     name   categories        lat        lng
0           Alley Theatre      Theater  29.761671 -95.365313
1  Wortham Theater Center      Theater  29.763353 -95.365663
2            Conservatory  Beer Garden  29.760427 -95.361570
3          House of Blues  Music Venue  29.753822 -95.363953
4      Buffalo Bayou Walk        Trail  29.762177 -95.375844


In [9]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [10]:

# create map of Houston venues using latitude and longitude values
map_hou = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill= True,
        fill_color='lightblue',
        fill_opacity=0.7,
    ).add_to(map_hou)  
    
map_hou

## INFORMATION ABOUT PUBLIC TRANSPORTATION

For the location of public transportation centers I will use the location of METRORail stations.
There are several MetroRail lines, so dataframes will be extracted from the web separately, treated and then the separate Dataframes will be merged.
Finally, the merged df will be treated with Geopy so we get the location coordinates (latitude, longitidue) of all the stations, regardless of the line they belong to.

In [11]:
pd_redline = pd.read_html('https://www.ridemetro.org/Pages/RedLine.aspx')[0]
pd_redline = pd_redline.drop(pd_redline.columns[[1, 3]], axis=1)
pd_redline['Rail Line']='Red Line'
pd_redline.head(10)

,Location,Address,Rail Line
0,Northline Transit Center / HCC,"7850 Fulton St., Houston, TX 77022",Red Line
1,Melbourne / North Lindale,"6350 Fulton St., Houston, TX 77022",Red Line
2,Lindale Park,"5350 Fulton St., Houston, TX 77009",Red Line
3,Cavalcade,"4750 Fulton St., Houston, TX 77009",Red Line
4,Moody Park,"4050 Fulton St., Houston, TX 77009",Red Line
5,Fulton / North Central,"2650 Fulton St., Houston, TX 77009",Red Line
6,Quitman / Near Northside,"2350 N Main St., Houston, TX 77009",Red Line
7,Burnett Transit Center / Casa de Amigos,"1450 N Main St., Houston TX 77009",Red Line
8,UH-Downtown,"5 1/2 Main St., Houston, TX 77002",Red Line
9,Preston,"311 1/2 Main St., Houston, TX 77002 450 1/2 M...",Red Line


In [12]:
pd_greenline = pd.read_html('https://www.ridemetro.org/Pages/GreenLine.aspx')[0]
pd_greenline = pd_greenline.drop(pd_greenline.columns[[1, 3]], axis=1)
pd_greenline['Rail Line']='Green Line'
pd_greenline.head(10)


,Location,Address,Rail Line
0,Theater District,550 Rusk St. 650 Capitol St.,Green Line
1,Central Station,1150 Rusk St. 1150 Capitol St.,Green Line
2,Convention District,1715 Rusk St. 1615 Capitol St.,Green Line
3,EaDo / Stadium,2020 1/2 Texas Ave.,Green Line
4,Coffee Plant / Second Ward,3500 1/2 Harrisburg Blvd.,Green Line
5,Lockwood / Eastwood,4535 1/2 Harrisburg Blvd. 4601 1/2 Harrisburg...,Green Line
6,Altic / Howard Hughes,5500 1/2 Harrisburg Blvd.,Green Line
7,Cesar Chavez / 67th St.,6700 1/2 Harrisburg Blvd.,Green Line
8,Magnolia Park Transit Center,7000 1/2 Harrisburg Blvd.,Green Line


In [13]:
pd_purpleline = pd.read_html('https://www.ridemetro.org/Pages/PurpleLine.aspx')[0]
pd_purpleline = pd_purpleline.drop(pd_purpleline.columns[[1, 3]], axis=1)
pd_purpleline['Rail Line']='Purple Line'
pd_purpleline.head(10)

,Location,Address,Rail Line
0,Theater District,550 Rusk St. 650 Capitol St.,Purple Line
1,Central Station,1150 Rusk 1150 Capitol St.,Purple Line
2,Convention District,1715 Rusk St. 1615 Capitol St.,Purple Line
3,EaDo / Stadium,2020 1/2 Texas Ave.,Purple Line
4,Leeland / Third Ward,1550 Scott St. 1650 Scott St.,Purple Line
5,Elgin / Third Ward,3250 Scott St.,Purple Line
6,TSU / UH Athletics District,3715 Scott St.,Purple Line
7,UH South / University Oaks,4450 Wheeler St. 4451 Wheeler St.,Purple Line
8,"MacGregor Park / Martin Luther King, Jr.",4750 1/2 Martin Luther King Blvd. 4751 1/2 Ma...,Purple Line
9,Palm Center Transit Center,5450 Griggs St.,Purple Line


In [14]:
# Stack the DataFrames on top of each other
pd_metroRail = pd.concat([pd_redline, pd_greenline, pd_purpleline], axis=0)
pd_metroRail= pd_metroRail.reset_index(drop=True)
pd_metroRail

,Location,Address,Rail Line
0,Northline Transit Center / HCC,"7850 Fulton St., Houston, TX 77022",Red Line
1,Melbourne / North Lindale,"6350 Fulton St., Houston, TX 77022",Red Line
2,Lindale Park,"5350 Fulton St., Houston, TX 77009",Red Line
3,Cavalcade,"4750 Fulton St., Houston, TX 77009",Red Line
4,Moody Park,"4050 Fulton St., Houston, TX 77009",Red Line
5,Fulton / North Central,"2650 Fulton St., Houston, TX 77009",Red Line
6,Quitman / Near Northside,"2350 N Main St., Houston, TX 77009",Red Line
7,Burnett Transit Center / Casa de Amigos,"1450 N Main St., Houston TX 77009",Red Line
8,UH-Downtown,"5 1/2 Main St., Houston, TX 77002",Red Line
9,Preston,"311 1/2 Main St., Houston, TX 77002 450 1/2 M...",Red Line


In [15]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import parser,urllib.request,requests # library to handle requests
from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
pd_metroRail['location'] = pd_metroRail['Address'].apply(geocode)
pd_metroRail['point'] = pd_metroRail['location'].apply(lambda loc: tuple(loc.point) if loc else None)
pd_metroRail[['latitude', 'longitude', 'altitude']] = pd.DataFrame(pd_metroRail['point'].tolist(), index=pd_metroRail.index)

pd_metroRail = pd_metroRail.drop(pd_metroRail.columns[[3, 4, 7]], axis=1)

pd_metroRail = pd_metroRail.dropna()

pd_metroRail

,Location,Address,Rail Line,latitude,longitude
0,Northline Transit Center / HCC,"7850 Fulton St., Houston, TX 77022",Red Line,29.832373,-95.375859
1,Melbourne / North Lindale,"6350 Fulton St., Houston, TX 77022",Red Line,29.834503,-95.375770
2,Lindale Park,"5350 Fulton St., Houston, TX 77009",Red Line,29.808188,-95.369287
3,Cavalcade,"4750 Fulton St., Houston, TX 77009",Red Line,29.802894,-95.368086
4,Moody Park,"4050 Fulton St., Houston, TX 77009",Red Line,29.796599,-95.367282
5,Fulton / North Central,"2650 Fulton St., Houston, TX 77009",Red Line,29.785800,-95.360636
6,Quitman / Near Northside,"2350 N Main St., Houston, TX 77009",Red Line,29.781820,-95.364257
7,Burnett Transit Center / Casa de Amigos,"1450 N Main St., Houston TX 77009",Red Line,29.772269,-95.359689
8,UH-Downtown,"5 1/2 Main St., Houston, TX 77002",Red Line,29.757578,-95.364352
10,Central Station Main,"715 1/2 Main St., Houston, TX 77002",Red Line,29.757578,-95.364352


Unfortunately, geopy failed to retrieve the coordinates from the addresses of several Stations. Those stations were dropped.
This is an error that will surely change the results found in the project. 

### Create map of the location of the METRORail stations in Houston  using latitude and longitude values

In [16]:

map_MR_hou = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(pd_metroRail['latitude'], pd_metroRail['longitude'], pd_metroRail['Location']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        radius=10,
        popup=label,
        num_sides = 4, 
        color='darkpurple',
        fill= True,
        fill_color='purple',
        fill_opacity=0.8,
    ).add_to(map_MR_hou)  
    
map_MR_hou

## INFORMATION ABOUT HOSPITALS

The database for this section was retreieved from the Texas Health and Human Services website. The data used is the **2019 DSHS/AHA/THA Annual Survey of Hospitals.**
Data was transformed from a PDF to a CSV file, retrieved as a Pandas df, and treated to show only the Hospitals in Houston. 

In [17]:
# The code was removed by Watson Studio for sharing.

,Facility,Address,City,County,Zip,Name,Phone,Beds,Beds.1,Ownership
0,Abilene Regional Medical Center,6250 Highway 83/84,Abilene,TAYLOR,79606-5299,Mr. Michael D. Murphy,(325)428-1010,231,0,Corporation
1,ContinueCARE Hospital at Hendrick Medical Center,"1900 Pine, 7th Floor",Abilene,TAYLOR,79601-2432,Billy Blasingame,(325)670-2966,23,0,Other NFP
2,Encompass Health Rehabilitation Hospital of Ab...,6401 Directors Parkway,Abilene,TAYLOR,79606,Mr. Joe Roberson,(325) 691-1600,30,NaN,Partnership
3,Hendrick Medical Center,1900 Pine,Abilene,TAYLOR,79601-2432,Brad Holland,(325)670-2201,540,0,Church
4,Methodist Hospital for Surgery,17101 N. Dallas Parkway,Addison,DALLAS,75001-7103,Trey,(469)248-3904,32,NaN,Partnership
5,CHRISTUS Spohn Hospital Alice,2500 East Main,Alice,JIM WELLS,78332-4169,Mr. David LeMonte,(361)661-8016,135,0,Church
6,PAM Rehabilitation Hospital of Allen,1001 Raintree Circle,Allen,COLLIN,75013,Mr. Mark Senko,(972)908-2017,40,NaN,Partnership
7,Texas Health Presbyterian Hospital Allen,1105 Central Expressway North,Allen,COLLIN,75013-6103,Mr. Jared Shelton,(972)747-6226,68,NaN,Other NFP
8,Big Bend Regional Medical Center,2600 Highway 118 North,Alpine,BREWSTER,79830-2002,Ms. Diane Moore,(432)837-3447,25,0,Corporation
9,Baptist St. Anthony's Hospital,1600 Wallace Boulevard,Amarillo,POTTER,79106-1799,Mr. Bob Williams,(806)212-5801,445,0,Corporation


In [18]:
df_hospitals = df_hospitals.drop(df_hospitals.columns[[3, 5, 6, 7 ,8 ,9]], axis=1)
df_hospitals.head(10)

,Facility,Address,City,Zip
0,Abilene Regional Medical Center,6250 Highway 83/84,Abilene,79606-5299
1,ContinueCARE Hospital at Hendrick Medical Center,"1900 Pine, 7th Floor",Abilene,79601-2432
2,Encompass Health Rehabilitation Hospital of Ab...,6401 Directors Parkway,Abilene,79606
3,Hendrick Medical Center,1900 Pine,Abilene,79601-2432
4,Methodist Hospital for Surgery,17101 N. Dallas Parkway,Addison,75001-7103
5,CHRISTUS Spohn Hospital Alice,2500 East Main,Alice,78332-4169
6,PAM Rehabilitation Hospital of Allen,1001 Raintree Circle,Allen,75013
7,Texas Health Presbyterian Hospital Allen,1105 Central Expressway North,Allen,75013-6103
8,Big Bend Regional Medical Center,2600 Highway 118 North,Alpine,79830-2002
9,Baptist St. Anthony's Hospital,1600 Wallace Boulevard,Amarillo,79106-1799


In [19]:
df_hospitals = df_hospitals[df_hospitals.City == 'Houston']
df_hospitals= df_hospitals.reset_index(drop=True)
df_hospitals.head(10)

,Facility,Address,City,Zip
0,AD Hospital East LLC,12950 EAST FREEWAY SUITE 100,Houston,77015-5710
1,"Altus Houston Hospital, Celestial Hospital, Od...",6011 W Sam Houston Parkway S,Houston,77072
2,Behavioral Hospital of Bellaire,5314 Dashwood,Houston,77081
3,CHI St Luke's Health Baylor College of Medicin...,6720 Bertner Street,Houston,77030-2697
4,Cornerstone Specialty Hospitals Medical Center,2001 Hermann Drive,Houston,77004
5,Cypress Creek Hospital,17750 Cali Drive,Houston,77090-2700
6,Encompass Health Rehabilitation Hospital of Cy...,13031 Wortham Center Drive,Houston,77065-5662
7,Encompass Health Rehabilitation Hospital The V...,20180 Chasewood Park Drive,Houston,77070
8,First Texas Hospital,9922 Louetta Road,Houston,77070
9,Harris County Psychiatric Center,2800 South MacGregor Way,Houston,77021-1000


In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="diego.gonzalez_13@hotmail.com")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_hospitals['location'] = df_hospitals['Address'].apply(geocode)

df_hospitals['point'] = df_hospitals['location'].apply(lambda loc: tuple(loc.point) if loc else None)


df_hospitals.head(10)

,Facility,Address,City,Zip,location,point
0,AD Hospital East LLC,12950 EAST FREEWAY SUITE 100,Houston,77015-5710,None,None
1,"Altus Houston Hospital, Celestial Hospital, Od...",6011 W Sam Houston Parkway S,Houston,77072,"(West Sam Houston Parkway South, Bellaire West...","(29.6742523, -95.5605107, 0.0)"
2,Behavioral Hospital of Bellaire,5314 Dashwood,Houston,77081,"(Dashwood, Huron County, Southwestern Ontario,...","(43.3460366, -81.6352066, 0.0)"
3,CHI St Luke's Health Baylor College of Medicin...,6720 Bertner Street,Houston,77030-2697,"(Bertner Street, Westbrook, Mitchell County, T...","(32.3601806, -101.0091135, 0.0)"
4,Cornerstone Specialty Hospitals Medical Center,2001 Hermann Drive,Houston,77004,"(2001, Hermann Drive, Houston, Harris County, ...","(29.717511181818182, -95.38278336363636, 0.0)"
5,Cypress Creek Hospital,17750 Cali Drive,Houston,77090-2700,"(17750, Cali Drive, Cypress Forest Lakes, Bamm...","(30.02624858428939, -95.44734458919896, 0.0)"
6,Encompass Health Rehabilitation Hospital of Cy...,13031 Wortham Center Drive,Houston,77065-5662,"(Wortham Center Drive, Harris County, Texas, 7...","(29.9215174, -95.6121834, 0.0)"
7,Encompass Health Rehabilitation Hospital The V...,20180 Chasewood Park Drive,Houston,77070,"(20180, Chasewood Park Drive, Louetta, Kohrvil...","(29.990145571428574, -95.57030385714286, 0.0)"
8,First Texas Hospital,9922 Louetta Road,Houston,77070,"(Louetta Road, Houston, Harris County, Texas, ...","(29.9967191, -95.600065, 0.0)"
9,Harris County Psychiatric Center,2800 South MacGregor Way,Houston,77021-1000,"(UTHealth Harris County Psychiatric Center, 28...","(29.71215455, -95.3745266544489, 0.0)"


In [21]:
df_hospitals = df_hospitals.replace(to_replace='None', value=np.nan).dropna()

df_hospitals[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_hospitals['point'].tolist(), index=df_hospitals.index)

df_hospitals = df_hospitals.drop(df_hospitals.columns[[4, 5, 8]], axis=1)
df_hospitals.head(10)

,Facility,Address,City,Zip,latitude,longitude
1,"Altus Houston Hospital, Celestial Hospital, Od...",6011 W Sam Houston Parkway S,Houston,77072,29.674252,-95.560511
2,Behavioral Hospital of Bellaire,5314 Dashwood,Houston,77081,43.346037,-81.635207
3,CHI St Luke's Health Baylor College of Medicin...,6720 Bertner Street,Houston,77030-2697,32.360181,-101.009113
4,Cornerstone Specialty Hospitals Medical Center,2001 Hermann Drive,Houston,77004,29.717511,-95.382783
5,Cypress Creek Hospital,17750 Cali Drive,Houston,77090-2700,30.026249,-95.447345
6,Encompass Health Rehabilitation Hospital of Cy...,13031 Wortham Center Drive,Houston,77065-5662,29.921517,-95.612183
7,Encompass Health Rehabilitation Hospital The V...,20180 Chasewood Park Drive,Houston,77070,29.990146,-95.570304
8,First Texas Hospital,9922 Louetta Road,Houston,77070,29.996719,-95.600065
9,Harris County Psychiatric Center,2800 South MacGregor Way,Houston,77021-1000,29.712155,-95.374527
10,Harris Health System Ben Taub Hospital,1504 Taub Loop,Houston,77030-1608,29.710611,-95.393429


### Create a map of the Hospitals in Houston

In [22]:
latitude = 29.7589382 
longitude = -95.3676974

map_hosp_hou = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_hospitals['latitude'], df_hospitals['longitude'], df_hospitals['Address']):
    label = folium.Popup(label, parse_html=True)
    icon = folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='red',icon='medkit', prefix='fa')
    ).add_to(map_hosp_hou)  


map_hosp_hou

# INFORMATION ABOUT UNIVERSITIES IN HOUSTON
This database was retrieved from 'Unipage', the only relevant information is the name of the university; so there the data will be cleaned to find its location and present a Map with Folium


In [23]:
url = 'https://www.unipage.net/en/universities/houston'
r = requests.get(url)
df_universities = pd.read_html(r.text)[0]
df_universities

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Title,Bachelor,Master
0,NaN,93.0,37.0,Rice University,"39,000 USD","39,000 USD"
1,NaN,360.0,90.0,University of Houston,"12,500 USD","9,000 USD"
2,NaN,565.0,118.0,Baylor College of Medicine,"19,000 USD","21,000 USD"
3,NaN,575.0,124.0,University of Texas M. D. Anderson Cancer Center,"17,500 USD","17,500 USD"
4,NaN,648.0,140.0,The University of Texas MD Anderson Cancer Center,"20,000 USD","20,000 USD"
5,NaN,1306.0,275.0,Houston Community College System,NaN,NaN
6,NaN,1685.0,368.0,Sam Houston State University,"15,000 USD","15,000 USD"
7,NaN,NaN,511.0,University of Houston-Downtown,"12,500 USD","12,500 USD"
8,NaN,NaN,555.0,University of Houston-Clear Lake,"12,500 USD","12,500 USD"
9,NaN,NaN,788.0,South Texas College of Law,NaN,NaN


In [24]:
df_universities = df_universities.drop(df_universities.columns[[0, 1, 2, 4, 5]], axis=1)
df_universities.columns = ['University']

df_universities.head(10)

,University
0,Rice University
1,University of Houston
2,Baylor College of Medicine
3,University of Texas M. D. Anderson Cancer Center
4,The University of Texas MD Anderson Cancer Center
5,Houston Community College System
6,Sam Houston State University
7,University of Houston-Downtown
8,University of Houston-Clear Lake
9,South Texas College of Law


In [25]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="diego.gonzalez_13@hotmail.com")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_universities['location'] = (df_universities['University'] + ", Houston, TX").apply(geocode)

df_universities['point'] = df_universities['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_universities[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_universities['point'].tolist(), index=df_universities.index)

df_universities.head(10)

,University,location,point,latitude,longitude,altitude
0,Rice University,"(Rice University, Whitley Street, South Lawn, ...","(29.716791450000002, -95.40478113393792, 0.0)",29.716791,-95.404781,0.0
1,University of Houston,"(University of Houston, 4800, Calhoun Road, Fo...","(29.7207902, -95.34406271491369, 0.0)",29.720790,-95.344063,0.0
2,Baylor College of Medicine,"(Baylor College of Medicine, 1, Baylor Plaza, ...","(29.7104775, -95.39580027495086, 0.0)",29.710477,-95.395800,0.0
3,University of Texas M. D. Anderson Cancer Center,None,None,NaN,NaN,NaN
4,The University of Texas MD Anderson Cancer Center,(University of Texas MD Anderson Cancer Center...,"(29.70766895, -95.39762257223782, 0.0)",29.707669,-95.397623,0.0
5,Houston Community College System,None,None,NaN,NaN,NaN
6,Sam Houston State University,None,None,NaN,NaN,NaN
7,University of Houston-Downtown,"(University of Houston-Downtown, 1, Main Stree...","(29.7661239, -95.35963162932202, 0.0)",29.766124,-95.359632,0.0
8,University of Houston-Clear Lake,None,None,NaN,NaN,NaN
9,South Texas College of Law,None,None,NaN,NaN,NaN


In [26]:
df_universities = df_universities.replace(to_replace='None', value=np.nan).dropna()

df_universities = df_universities.drop(df_universities.columns[[1, 2, 5]], axis=1)
df_universities.head(10)

,University,latitude,longitude
0,Rice University,29.716791,-95.404781
1,University of Houston,29.720790,-95.344063
2,Baylor College of Medicine,29.710477,-95.395800
4,The University of Texas MD Anderson Cancer Center,29.707669,-95.397623
7,University of Houston-Downtown,29.766124,-95.359632
10,University of Houston–Victoria,29.720790,-95.344063
11,Texas Southern University,29.720906,-95.359620
12,University of St. Thomas,29.736824,-95.394147
13,Houston Baptist University,29.694429,-95.514756
15,Center for Advanced Legal Studies,29.731300,-95.419300


### CREATE A MAP OF UNIVERSITIES IN HOUSTON

In [27]:
latitudeHou = 29.7589382
longitudeHou = -95.3676974

map_univ_hou = folium.Map(location=[latitudeHou, longitudeHou], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_universities['latitude'], df_universities['longitude'], df_universities['University']):
    label = folium.Popup(label, parse_html=True)
    icon = folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='blue',icon='university', prefix='fa')
    ).add_to(map_univ_hou)  


map_univ_hou

# CREATE A MAP THAT INCLUDES ALL FORMER MAPS

To cluster all the previous locations inside neighborhoods I loaded a CSV containing the neighborhoods in Houston with their respective Zip Code

In [28]:

body = client_39203f0b5190433ab09b03ce75051b7d.get_object(Bucket='capstoneprojectthebattleofneighbo-donotdelete-pr-61axpvxzlsmhix',Key='Houston Neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )


df_neighborhood = pd.read_csv(body, encoding= 'unicode_escape')
df_neighborhood.head()


,Neighborhood Name,Zip Code
0,Willowbrook,77035
1,Greater Greenspoint,77037
2,Carverdale,77041
3,Fairbanks - Northwest Crossing,77040
4,Greater Inwood,77088


In [29]:
!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df_neighborhood.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

df_neighborhood["Latitude"] = latitude
df_neighborhood["Longitude"] = longitude

df_neighborhood.head()

     |████████████████████████████████| 378kB 8.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 24.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 37.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 17.0MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/b2/31/85/71db2edd0f7b97ad8fd9de009b8ab2e6a98064595d74aff18e
Successfully built autopep8
ERROR: spyder 3.3.3 requires pyqt5<=5.12; python_version >= "3", which is not installed.
  Found existing installation: pycodestyle 2.5.0
    Uninstalling pycodestyle-2.5.0:
      Successfully uninstalled pycodestyle-2.5.0
Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


,Neighborhood Name,Zip Code,Latitude,Longitude
0,Willowbrook,77035,29.65,-95.48
1,Greater Greenspoint,77037,29.90,-95.39
2,Carverdale,77041,29.87,-95.57
3,Fairbanks - Northwest Crossing,77040,29.87,-95.54
4,Greater Inwood,77088,29.88,-95.45


In [30]:
import geopandas as gpd
import requests
data = requests.get("https://cohegis.houstontx.gov/cohgispub/rest/services/PD/Government_Boundaries_wm/MapServer/3?f=pjson")
gdf = gpd.GeoDataFrame(data.json())
gdf.head()
#UNABLE TO GET JSON FILE 

ValueError: arrays must all be same length

In [31]:
latitudeHou = 29.7589382
longitudeHou = -95.3676974

finalMap = folium.Map(location=[latitudeHou, longitudeHou], zoom_start=13)


# NEIGHBORHOODS MAP
for lat, lng, label in zip(df_neighborhood['Latitude'], df_neighborhood['Longitude'], df_neighborhood['Neighborhood Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=100,
        popup=label,
        color='red',
        fill= True,
        fill_color='orange',
        fill_opacity=0.5,
    ).add_to(finalMap)  


# VENUES MAP
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    icon = folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='green',icon='popcorn', prefix='fa')
    ).add_to(finalMap)  
    

# METRORAIL STATIONS MAP
for lat, lng, label in zip(pd_metroRail['latitude'], pd_metroRail['longitude'], pd_metroRail['Location']):
    label = folium.Popup(label, parse_html=True)
    icon = folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='purple',icon='subway', prefix='fa')
    ).add_to(finalMap)  

# HOSPITALS MAP
for lat, lng, label in zip(df_hospitals['latitude'], df_hospitals['longitude'], df_hospitals['Address']):
    label = folium.Popup(label, parse_html=True)
    icon = folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='red',icon='medkit', prefix='fa')
    ).add_to(finalMap) 
    
# UNIVERSITIES MAP
for lat, lng, label in zip(df_universities['latitude'], df_universities['longitude'], df_universities['University']):
    label = folium.Popup(label, parse_html=True)
    icon = folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='blue',icon='university', prefix='fa')
    ).add_to(finalMap)  


finalMap

## LIMITATIONS
There were several limitations on the development of this work:
1. Limitation in the number of venues that can be searched via the Foursquare API. This will show only a big cluster of venues near the center point, without showing the venues that are farther away.
2. On the cleanse part of the METRORail Stations, there were several stations which were 'dropped' since geopandas was unable to retrieve coordinates. 
3. I was unable to gather accurate information from a GEOJSON file to show the exact shape of the neighborhoods; nevertheless we can get a somewhat accurate approximation of their location. 

## CONCLUSIONS
Taking into consideration the limitations and problems I encountered while developing my project, we can still draw some important conslusions:

* As we can expect many there are many important venues and ammenities located near the center of the city, Downtown Houston is a very attractive place to live. Nevertheless, we can expect higher apartment and housing prices in such location.
* There are a lot of METRORail stations in all of Houston, so if proximity or closeness to one of this stations is the most important point for your decision, you really have many neighborhoods to consider. This makes Houston a really attractive city.
* Hospitals have a lot of dispersion on the city, nevertheless, there is still a higher amount of hospitals clustered near the Downtown Houston.
* A really high number of Universities are located near the center of the city; moreso, they are located Southwest of Downtown Houston.



### Final Questions to Answer:

#### Are there attractive venues nearby?
There are many attractive venues spreaded through Houston, although we were limited to only 100 venues, we covered a lot of terrain. Also, it is important to consider that due to the important and numerous pressence of public transportation stations on Houston, proximity to a desired venue will not become an issue for people living in Houston.  
#### Are there any patterns for the distribution of popular venues and public services across Houston?
Yes, based on our research, there is an important cluster of venues located around public services in the city. We can expect the same to hold true if we go further away from Downtown Houston. 
#### Final recomendations from the information gathered.
My recommendation would be to live near Downtown Houston, nevertheless to avoid living in such neighborhood. I would decide to live either on Neartown - Montrose or MacGregor neighborhoods. They both have all the attractive features which I desire and described at the beginning of this project (although no METRORail Stations appear on the map, they do exist on such clusters). Also, the cost to live in such neighborhoods will be somewhat lower than living on Downtown Houston, plus being less overcrowded. 